In [6]:
import requests
from typing import Any
from tqdm import tqdm
def get_neuronpedia_feature(
    # feature: int, 
    layer: int, 
    # model: str = "gpt2-small"
    #  , dataset: str = "res-jb"
) -> dict[str, Any]:
    """Fetch a feature from Neuronpedia API."""
    length = 24576
    layer_activation_length = []
    for feature in tqdm(range(0, length)):
        url = f"https://www.neuronpedia.org/api/feature/gpt2-small/{layer}-res-jb/{feature}"
        result = requests.get(url).json()
        l = int(len(result["activations"]))
        layer_activation_length.append(l)
    return layer_activation_length

In [7]:
layer = []
for layer in range(12):
    layer_activation_length = get_neuronpedia_feature(layer)
    layer.append(layer_activation_length)
    print(layer_activation_length)

  0%|          | 37/24576 [00:12<2:22:53,  2.86it/s]


KeyboardInterrupt: 